In [ ]:
import os
import numpy as np
import pandas as pd
import scipy
from scipy.io import loadmat
import matplotlib.pyplot as plt
import seaborn as sns
import shutil

In [ ]:
path = "../input/battery-data-set"
dis = os.listdir(path)
dis_mat = []
battery_grp = {
    
}
for i in dis :
    filtered_list = list(filter(lambda x:x.split('.')[-1] == 'mat',os.listdir(f"{path}/{i}")))
    battery_grp[i.split('BatteryAgingARC')[-1][1:]] = list(map(lambda x:x.split('.')[0],filtered_list))
    dis_mat.extend(list(map(lambda x:f"{path}/{i}/{x}",filtered_list)))

battery_grp['5_6_7_18'] = battery_grp['FY08Q4']
del battery_grp['FY08Q4']

dis_mat
battery_grp

In [ ]:
from scipy.io import loadmat

bs = [x.split('/')[-1].split('.')[0] for x in dis_mat]
print(bs)

ds = []
for b in dis_mat:
    ds.append(loadmat(b))

types = []
times = []
ambient_temperatures = []
datas = []

for i in range(len(ds)):
    x = ds[i][bs[i]]["cycle"][0][0][0]
#     ambient_temperatures.append(x['ambient_temperature'])
    ambient_temperatures.append(list(map(lambda y:y[0][0],x['ambient_temperature'])))
    types.append(x['type'])
    times.append(x['time'])
    datas.append(x['data']) 
    
batteries = []
cycles = []
for i in range(len(ds)):
    batteries.append(bs[i])
    cycles.append(datas[i].size)
    
battery_cycle_df = pd.DataFrame({'Battery':batteries,'Cycle':cycles}).sort_values('Battery',ascending=True)
battery_cycle_df.drop_duplicates(inplace=True)
battery_cycle_df

In [ ]:
Cycles = {}
params = ['Voltage_measured','Current_measured','Temperature_measured',
          'Current_load','Voltage_load','Time','Capacity',]

for i in range(len(bs)):
    Cycles[bs[i]] = {}
    Cycles[bs[i]]['count'] = 0 # This is true for battery B0005, 06, 07
#     ambient_temperatures[i]
    for param in params:
        Cycles[bs[i]][param] = []
        for j in range(datas[i].size):
            if types[i][j] == 'discharge':
                Cycles[bs[i]][param].append(datas[i][j][param][0][0][0])
        
    cap = []
    amb_temp = []
    for j in range(datas[i].size):
        if types[i][j] == 'discharge':
            cap.append(datas[i][j]['Capacity'][0][0][0])
            amb_temp.append(ambient_temperatures[i][j])
            
    Cycles[bs[i]]['Capacity'] = np.array(cap)
    Cycles[bs[i]]['ambient_temperatures'] = np.array(amb_temp)
Cycles = pd.DataFrame(Cycles)

battery_count = []
for b in battery_cycle_df['Battery'].values.tolist() :
    Cycles[b]['count'] = len(Cycles[b]['Capacity'])
Cycles

In [ ]:
Cycles['B0052']

In [ ]:
Cycles.loc['count',:].reset_index()
battery_cycle_df
compare_df = Cycles.loc['count',:].reset_index().set_index('index').join(battery_cycle_df.set_index('Battery')).rename(columns={'count':'After','Cycle':'Before'})
compare_df[['Before','After']]

In [ ]:
pd.DataFrame(Cycles['B0047'])

In [ ]:
df = pd.DataFrame({})
cols = ['Voltage_measured','Current_measured','Temperature_measured',
          'Current_load','Voltage_load','Time','Capacity','ambient_temperatures']
for col in cols :
    df[col] = Cycles['B0047'][col]
max_len = np.max(df['Time'].apply(lambda x:len(x)).values)
df_x = df.drop(columns=['Capacity']).values
df_y = df['Capacity'].values
df

In [ ]:
batteries

In [ ]:
df_all = pd.DataFrame({})
max_len = 0

experiment1 = ["B0005", "B0006", "B0007", "B0018"]
experiment2 = ["B0025", "B0026", "B0027", "B0028"]
experiment3 = ["B0029", "B0030", "B0031", "B0032"]
experiment4 = ["B0033", "B0034", "B0036"]
experiment5 = ["B0038", "B0039", "B0040"]
experiment6 = ["B0041", "B0042", "B0043", "B0044"]
experiment7 = ["B0045", "B0046", "B0047", "B0048"]
experiment8 = ["B0049", "B0050", "B0051", "B0052"]
experiment9 = ["B0053", "B0054", "B0055", "B0056"]

exp_try_out = experiment1 # replace karke wo experiment wala dalo

for bat in exp_try_out:
    df = pd.DataFrame({})
    cols = ['Voltage_measured','Current_measured','Temperature_measured',
              'Current_load','Voltage_load','Time','Capacity','ambient_temperatures']
    for col in cols :
        df[col] = Cycles[bat][col]
    max_l = np.max(df['Time'].apply(lambda x:len(x)).values)
    max_len = max(max_l,max_len)
    df_all = pd.concat([df_all,df],ignore_index=True)
    
df=df_all.reset_index(drop=True)
df

In [ ]:
flag = False
for i,j in enumerate(df['Capacity']) :
    try :
        if len(j) :
            df['Capacity'][i] = j[0] 
        else :
            df['Capacity'][i] = 0
        flag = True
    except :
        pass
print(flag)

In [ ]:
# df[1719:1780]

In [ ]:
df.shape,max_len

In [ ]:
def to_padded_numpy(l, shape):
    padded_array = np.zeros(shape)
#     print(l)
    padded_array[:len(l)] = l
    return padded_array

df_x = df.drop(columns=['Capacity','ambient_temperatures']).values
df_y = df['Capacity'].values
ambient_temperatures = df['ambient_temperatures'].values
n,m = df_x.shape[0], df_x.shape[1]
temp2 = np.zeros((n,m,max_len))
for i in range(n) :
    for j in range(m) :
        temp2[i][j] = to_padded_numpy(df_x[i][j],max_len)

df_x = temp2.reshape(n,m*max_len)
df_x = np.concatenate((df_x, ambient_temperatures.reshape(ambient_temperatures.shape[0],1)), axis=1)
df_x.shape

In [ ]:
split = 20
train_size = df_x.shape[0]*(100-split)//100
test_size = df_x.shape[0]-train_size

train_x,test_x = df_x[:train_size],df_x[train_size:]
train_y,test_y = df_y[:train_size],df_y[train_size:]
train_x.shape,test_x.shape,train_y.shape,test_y.shape

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_x,train_y)

In [ ]:
from sklearn.metrics import max_error,mean_absolute_error,mean_squared_error,mean_squared_log_error
from sklearn.metrics import median_absolute_error,mean_absolute_percentage_error,r2_score

def get_scores(y_true,y_pred) :
    return {
        'max_error':max_error(y_true,y_pred),
        'mean_absolute_error':mean_absolute_error(y_true,y_pred),
        'mean_squared_error':mean_squared_error(y_true,y_pred),
        'mean_squared_log_error':mean_squared_log_error(y_true,y_pred),
        'median_absolute_error':median_absolute_error(y_true,y_pred),
        'mean_absolute_percentage_error':mean_absolute_percentage_error(y_true,y_pred),
        'r2_score':r2_score(y_true,y_pred)
    }

def get_preds(model,data_x) :
    return model.predict(data_x).clip(min=0)

print(get_scores(train_y,get_preds(lr,train_x)))
print(get_scores(test_y,get_preds(lr,test_x)))

# for i,j in enumerate(lr.predict(train_x)) :
#     print(train_y[i],j)

In [ ]:
def to_padded_numpy(l, shape):
    padded_array = np.zeros(shape)
#     print(l)
    padded_array[:len(l)] = l
    return padded_array

df_x = df.drop(columns=['Capacity','ambient_temperatures']).values
df_y = df['Capacity'].values
ambient_temperatures = df['ambient_temperatures'].values
n,m = df_x.shape[0], df_x.shape[1]
temp2 = np.zeros((n,m,max_len))
for i in range(n) :
    for j in range(m) :
        temp2[i][j] = to_padded_numpy(df_x[i][j],max_len)

df_x = temp2
# df_x = temp2.reshape(n,m*max_len)
# df_x = np.concatenate((df_x, ambient_temperatures.reshape(ambient_temperatures.shape[0],1)), axis=1)
df_x.shape

test_split = 10
val_split = 10
train_size = df_x.shape[0]*(100-(test_split+val_split))//100
val_size = df_x.shape[0]*(val_split)//100
test_size = df_x.shape[0]-train_size-val_size

# train_x,test_x = df_x[:train_size],df_x[train_size:]
# train_y,test_y = df_y[:train_size],df_y[train_size:]

# train_x,train_y = df_x[:train_size],df_y[:train_size]
# val_x,val_y = df_x[train_size:train_size+val_size],df_y[train_size:train_size+val_size]
# test_x,test_y = df_x[train_size+val_size:],df_y[train_size+val_size:]

# train_x=np.asarray(train_x).astype('float32')
# test_x=np.asarray(test_x).astype('float32')
# train_y=np.asarray(train_y).astype('float32')
# test_y=np.asarray(test_y).astype('float32')

from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y = train_test_split(df_x,df_y,test_size=0.2,random_state=0)
test_x,val_x,test_y,val_y = train_test_split(test_x,test_y,test_size=0.5,random_state=0)

temp = train_x[0][0][0]
train_x,test_x,train_y,test_y = train_test_split(df_x,df_y,test_size=0.2,random_state=0)
test_x,val_x,test_y,val_y = train_test_split(test_x,test_y,test_size=0.5,random_state=0)

# ((508, 6, 371), (65, 6, 371), (508,), (65,))
train_x.shape,test_x.shape,train_y.shape,test_y.shape

In [ ]:
train_x[0][0][0]
assert train_x[0][0][0] == temp

In [ ]:
for i,j in enumerate(train_y) :
    try :
        len(j)
        print(i)
#         break
    except :
#         print(e)
#         break
        temp = 1

In [ ]:
import numpy as np
import pandas as pd
import scipy.io
import math
import os
import ntpath
import sys
import logging
import time
import sys
import random

from importlib import reload
import plotly.graph_objects as go

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import LSTM, Embedding, RepeatVector, TimeDistributed, Masking, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LambdaCallback

IS_TRAINING = True

# Original Parameters
# LEARNING_RATE = 0.000003
# REGULARIZATION = 0.0002
# NUM_EPOCHS = 500
# BATCH_SIZE = 32

LEARNING_RATE = 0.0007
REGULARIZATION = 0.0002
NUM_EPOCHS = 200
BATCH_SIZE = 64
EARLY_STOPPING=25
STEP_LR = LEARNING_RATE/100

if IS_TRAINING:
    EXPERIMENT = "lstm_rul_nasa_randomized"

    experiment_name = time.strftime("%Y-%m-%d-%H-%M-%S") + '_' + EXPERIMENT
    print(experiment_name)

    # Model definition

    opt = tf.keras.optimizers.Adam(lr=LEARNING_RATE)

    model = Sequential()
    model.add(Masking(input_shape=(train_x.shape[1], train_x.shape[2])))
#     model.add(LSTM(256, activation='selu',return_sequences=True,kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(LSTM(256, activation='selu',return_sequences=True,kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(LSTM(256, activation='selu',return_sequences=True,kernel_regularizer=regularizers.l2(REGULARIZATION)))
#     model.add(LSTM(128, activation='selu',return_sequences=True,kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(LSTM(128, activation='selu',return_sequences=True,kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(LSTM(128, activation='selu',return_sequences=True,kernel_regularizer=regularizers.l2(REGULARIZATION)))
#     model.add(LSTM(64, activation='selu',return_sequences=True,kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(LSTM(64, activation='selu',return_sequences=True,kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(LSTM(64, activation='selu',return_sequences=False,kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(Dense(64, activation='selu', kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(Dense(64, activation='selu', kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(Dense(32, activation='selu', kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(Dense(32, activation='selu', kernel_regularizer=regularizers.l2(REGULARIZATION)))
    model.add(Dense(1, activation='linear'))
    model.summary()
    
    model.compile(optimizer=opt, loss='huber', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    
data_path="./model"
if IS_TRAINING:
    def scheduler(epoch, lr):
        if epoch < 10:
            return lr+STEP_LR
        elif epoch%5 == 0:
            return lr*0.99
        return lr
    
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=EARLY_STOPPING)
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)    
    
    history = model.fit(train_x, train_y,epochs=NUM_EPOCHS, batch_size=BATCH_SIZE,verbose=1, validation_split=0.1,callbacks=[early_stopping,lr_scheduler])
    
    model.save(data_path + 'results/trained_model/%s.h5' % experiment_name)

    hist_df = pd.DataFrame(history.history)
    hist_csv_file = data_path + 'results/trained_model/%s_history.csv' % experiment_name
    with open(hist_csv_file, mode='w') as f:
        hist_df.to_csv(f)
    history = history.history

In [ ]:
model.evaluate(train_x, train_y)
model.evaluate(val_x, val_y)
model.evaluate(test_x, test_y)